# Thiết lập đường dẫn tổng

In [4]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.pardir))

print(sys.path)
# Thêm đường dẫn gốc vào sys.path nếu chưa có
# if project_root not in sys.path:
    # sys.path.append(project_root)

# Kiểm tra xem đã nhận chưa
print(f"Project Root: {project_root}")

['d:\\conda_envs\\machine_learning\\python310.zip', 'd:\\conda_envs\\machine_learning\\DLLs', 'd:\\conda_envs\\machine_learning\\lib', 'd:\\conda_envs\\machine_learning', '', 'd:\\conda_envs\\machine_learning\\lib\\site-packages', 'd:\\conda_envs\\machine_learning\\lib\\site-packages\\win32', 'd:\\conda_envs\\machine_learning\\lib\\site-packages\\win32\\lib', 'd:\\conda_envs\\machine_learning\\lib\\site-packages\\Pythonwin']
Project Root: d:\University\HK7\IT_Project\Project\Source


# Import các thư viện cần thiết

In [1]:
import pandas as pd
from utils.config import *
from utils.supporter import *
from data.datasets import *
from data.loader import get_data_loader
from engine.trainer import Trainer
from engine.evaluator import *

ModuleNotFoundError: No module named 'utils'

In [3]:
spatial_cfg = TrainingConfig('spatial', 'efficientnet')
spatial_cfg.NUM_EPOCHS = 5

freq_cfg = TrainingConfig('frequency', 'frequency')
freq_cfg.NUM_EPOCHS = 5

hybrid_cfg = TrainingConfig('hybrid', 'efficientnet_fft')
hybrid_cfg.NUM_EPOCHS = 5

In [4]:
# Load data
train_df = pd.read_csv(PathConfig.train_csv)
val_df = pd.read_csv(PathConfig.val_csv)
test_df = pd.read_csv(PathConfig.test_csv)

# Display first few rows of training data
train_df.head()

,path,label
0,..\..\Datasets\Real\13302.jpg,0
1,..\..\Datasets\AI_Generated\STARGAN_000641.jpg,1
2,..\..\Datasets\AI_Generated\STARGAN_001900.jpg,1
3,..\..\Datasets\Real\18920.jpg,0
4,..\..\Datasets\Real\1059.jpg,0


In [5]:
transforms_dict = get_transforms()

train_dataset = DeepFakeDataset(train_df, transform=transforms_dict['train'])
val_dataset = DeepFakeDataset(val_df, transform=transforms_dict['val'])
test_dataset = DeepFakeDataset(test_df, transform=transforms_dict['test'])

In [6]:
train_loader = get_data_loader(train_dataset, hybrid_cfg, True)
val_loader = get_data_loader(val_dataset, hybrid_cfg, False)
test_loader = get_data_loader(test_dataset, hybrid_cfg, False)

print("DataLoader đã được tạo thành công!")
print(f"Số batch trong tập train: {len(train_loader)}")
print(f"Số batch trong tập val: {len(val_loader)}")
print(f"Số batch trong tập test: {len(test_loader)}")

DataLoader đã được tạo thành công!
Số batch trong tập train: 744
Số batch trong tập val: 107
Số batch trong tập test: 213


In [7]:
for images, labels in train_loader:
    print(f"Kích thước batch ảnh: {images.size()}")
    print(f"Kích thước batch nhãn: {labels.size()}")
    break  # Chỉ kiểm tra batch đầu tiên

Kích thước batch ảnh: torch.Size([16, 3, 224, 224])
Kích thước batch nhãn: torch.Size([16])


In [13]:
spatial_model = build_model(spatial_cfg)

spatial_trainer = Trainer(
    spatial_model, train_loader, val_loader, spatial_cfg
)

🛠️ Building Model | Mode: spatial | Device: cuda


In [14]:
spatial_trainer.train()


STARTING TRAINING | Mode: spatial

Epoch 1/5


Validating: 100%|██████████| 107/107 [00:32<00:00,  3.26it/s]


   Train Loss: 0.0705 | Acc: 0.9738
   Val   Loss: 0.0199 | Acc: 0.9924
   New Best Val Acc: 0.9924
== Best model saved.

Epoch 2/5


Validating: 100%|██████████| 107/107 [00:32<00:00,  3.33it/s]


   Train Loss: 0.0207 | Acc: 0.9934
   Val   Loss: 0.0043 | Acc: 0.9982
   New Best Val Acc: 0.9982
== Best model saved.

Epoch 3/5


...

   Train Loss: 0.0133 | Acc: 0.9953
   Val   Loss: 0.0049 | Acc: 0.9982
   Epochs no improve: 1/10

Epoch 4/5


Validating: 100%|██████████| 107/107 [00:40<00:00,  2.67it/s]


   Train Loss: 0.0126 | Acc: 0.9965
   Val   Loss: 0.0046 | Acc: 0.9988
   New Best Val Acc: 0.9988
== Best model saved.

Epoch 5/5


Validating: 100%|██████████| 107/107 [00:39<00:00,  2.70it/s]

   Train Loss: 0.0099 | Acc: 0.9971
   Val   Loss: 0.0029 | Acc: 0.9988
   Epochs no improve: 1/10

Training Completed. Best Acc: 0.9988


In [19]:
plot_training_history(spatial_trainer.history, spatial_cfg.LOG_DIR)

📈 Đã lưu biểu đồ training tại: ..\..\Outputs\efficientnet\logs\training_history.png


In [16]:
evaluate_test_set(spatial_model, test_loader, spatial_cfg.DEVICE, spatial_cfg.LOG_DIR)


BẮT ĐẦU ĐÁNH GIÁ TRÊN TẬP TEST


Testing: 100%|██████████| 213/213 [01:32<00:00,  2.32it/s]



📊 KẾT QUẢ ĐÁNH GIÁ:
   Accuracy : 0.9988
   Precision: 1.0000
   Recall   : 0.9971
   F1 Score : 0.9986

📋 Chi tiết theo lớp:
              precision    recall  f1-score   support

        Real       1.00      1.00      1.00      2000
        Fake       1.00      1.00      1.00      1400

    accuracy                           1.00      3400
   macro avg       1.00      1.00      1.00      3400
weighted avg       1.00      1.00      1.00      3400

🔢 Đã lưu Confusion Matrix tại: ..\..\Outputs\efficientnet\logs\evaluation_results\confusion_matrix.png
📉 Đã lưu ROC Curve tại: ..\..\Outputs\efficientnet\logs\evaluation_results\roc_curve.png
=== ROC AUC Score: 1.0000

Đã lưu toàn bộ kết quả đánh giá tại: ..\..\Outputs\efficientnet\logs\evaluation_results


In [17]:
freq_model = build_model(freq_cfg)
freq_trainer = Trainer(
    freq_model, train_loader, val_loader, freq_cfg
)

freq_trainer.train()

🛠️ Building Model | Mode: frequency | Device: cuda

STARTING TRAINING | Mode: frequency

Epoch 1/5


Validating: 100%|██████████| 107/107 [00:37<00:00,  2.87it/s]


   Train Loss: 0.3838 | Acc: 0.8356
   Val   Loss: 0.3710 | Acc: 0.8559
   New Best Val Acc: 0.8559
== Best model saved.

Epoch 2/5


Validating: 100%|██████████| 107/107 [00:37<00:00,  2.86it/s]


   Train Loss: 0.3488 | Acc: 0.8524
   Val   Loss: 0.2648 | Acc: 0.8959
   New Best Val Acc: 0.8959
== Best model saved.

Epoch 3/5


...

   Train Loss: 0.3230 | Acc: 0.8654
   Val   Loss: 0.2501 | Acc: 0.9018
   New Best Val Acc: 0.9018
== Best model saved.

Epoch 4/5


Validating: 100%|██████████| 107/107 [00:32<00:00,  3.33it/s]


   Train Loss: 0.3081 | Acc: 0.8703
   Val   Loss: 0.2510 | Acc: 0.8988
   Epochs no improve: 1/10

Epoch 5/5


Validating: 100%|██████████| 107/107 [00:34<00:00,  3.10it/s]

   Train Loss: 0.2946 | Acc: 0.8781
   Val   Loss: 0.2932 | Acc: 0.8929
   Epochs no improve: 2/10

Training Completed. Best Acc: 0.9018


In [20]:
plot_training_history(freq_trainer.history, freq_cfg.LOG_DIR)

📈 Đã lưu biểu đồ training tại: ..\..\Outputs\frequency\logs\training_history.png


In [18]:
evaluate_test_set(freq_model, test_loader, freq_cfg.DEVICE, freq_cfg.LOG_DIR)


BẮT ĐẦU ĐÁNH GIÁ TRÊN TẬP TEST


Testing: 100%|██████████| 213/213 [01:15<00:00,  2.81it/s]



📊 KẾT QUẢ ĐÁNH GIÁ:
   Accuracy : 0.8950
   Precision: 0.9878
   Recall   : 0.7543
   F1 Score : 0.8554

📋 Chi tiết theo lớp:
              precision    recall  f1-score   support

        Real       0.85      0.99      0.92      2000
        Fake       0.99      0.75      0.86      1400

    accuracy                           0.90      3400
   macro avg       0.92      0.87      0.89      3400
weighted avg       0.91      0.90      0.89      3400

🔢 Đã lưu Confusion Matrix tại: ..\..\Outputs\frequency\logs\evaluation_results\confusion_matrix.png
📉 Đã lưu ROC Curve tại: ..\..\Outputs\frequency\logs\evaluation_results\roc_curve.png
=== ROC AUC Score: 0.9714

Đã lưu toàn bộ kết quả đánh giá tại: ..\..\Outputs\frequency\logs\evaluation_results


In [8]:
hybrid_model = build_model(hybrid_cfg)
hybrid_trainer = Trainer(
    hybrid_model, train_loader, val_loader, hybrid_cfg
)

hybrid_trainer.train()

🛠️ Building Model | Mode: hybrid | Device: cuda

STARTING TRAINING | Mode: hybrid

Epoch 1/5


Validating: 100%|██████████| 107/107 [00:48<00:00,  2.22it/s]


   Train Loss: 0.0649 | Acc: 0.9745
   Val   Loss: 0.0211 | Acc: 0.9929
   New Best Val Acc: 0.9929
== Best model saved.

Epoch 2/5


Validating: 100%|██████████| 107/107 [00:32<00:00,  3.25it/s]


   Train Loss: 0.0241 | Acc: 0.9913
   Val   Loss: 0.0073 | Acc: 0.9965
   New Best Val Acc: 0.9965
== Best model saved.

Epoch 3/5


...

   Train Loss: 0.0159 | Acc: 0.9946
   Val   Loss: 0.0085 | Acc: 0.9971
   New Best Val Acc: 0.9971
== Best model saved.

Epoch 4/5


Validating: 100%|██████████| 107/107 [00:31<00:00,  3.39it/s]


   Train Loss: 0.0163 | Acc: 0.9966
   Val   Loss: 0.0184 | Acc: 0.9947
   Epochs no improve: 1/10

Epoch 5/5


Validating: 100%|██████████| 107/107 [00:32<00:00,  3.34it/s]


   Train Loss: 0.0145 | Acc: 0.9956
   Val   Loss: 0.0070 | Acc: 0.9982
   New Best Val Acc: 0.9982
== Best model saved.

Training Completed. Best Acc: 0.9982


In [9]:
plot_training_history(hybrid_trainer.history, hybrid_cfg.LOG_DIR)

📈 Đã lưu biểu đồ training tại: ..\..\Outputs\efficientnet_fft\logs\training_history.png


In [10]:
evaluate_test_set(hybrid_model, test_loader, hybrid_cfg.DEVICE, hybrid_cfg.LOG_DIR)


BẮT ĐẦU ĐÁNH GIÁ TRÊN TẬP TEST


Testing: 100%|██████████| 213/213 [01:42<00:00,  2.09it/s]



📊 KẾT QUẢ ĐÁNH GIÁ:
   Accuracy : 0.9991
   Precision: 0.9979
   Recall   : 1.0000
   F1 Score : 0.9989

📋 Chi tiết theo lớp:
              precision    recall  f1-score   support

        Real       1.00      1.00      1.00      2000
        Fake       1.00      1.00      1.00      1400

    accuracy                           1.00      3400
   macro avg       1.00      1.00      1.00      3400
weighted avg       1.00      1.00      1.00      3400

🔢 Đã lưu Confusion Matrix tại: ..\..\Outputs\efficientnet_fft\logs\evaluation_results\confusion_matrix.png
📉 Đã lưu ROC Curve tại: ..\..\Outputs\efficientnet_fft\logs\evaluation_results\roc_curve.png
=== ROC AUC Score: 1.0000

Đã lưu toàn bộ kết quả đánh giá tại: ..\..\Outputs\efficientnet_fft\logs\evaluation_results
